## Train

In [ ]:
import os
import json
import torch
from logger import logger
from ultralytics import YOLO

WOW_VERSION = "classic"  # or "retail"
YOLO_VERSION = "yolo11m"  # or "yolo11s"
DIR_MODELS = "models"

# Check if ROCm-compatible GPU is available
logger.info(f"PyTorch version: {torch.__version__}")
logger.info(f"GPU available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    logger.info(f"Device count: {torch.cuda.device_count()}")
    logger.info(f"Current device index: {torch.cuda.current_device()}")
    logger.info(f"Device name: {torch.cuda.get_device_name(0)}")
    logger.info(f"Device capability: {torch.cuda.get_device_capability(0)}")
else:
    logger.warning(f"No compatible GPU detected use CPU.")

with open("config.json", "r") as f:
    config = json.load(f)
os.chdir(config["workdir"])
model_name = f"{YOLO_VERSION}-wow-fishing-{WOW_VERSION}"
model = YOLO(os.path.join(DIR_MODELS, f"{YOLO_VERSION}.pt"))
model.train(data="yolo-config.yaml", epochs=50, imgsz=960, device="0", batch=4, name=model_name, project=DIR_MODELS, exist_ok=True)
model.save(os.path.join(DIR_MODELS, f"{model_name}.pt"))
logger.info(f"Done.")

## Train-Val-Test Split

In [2]:
import random
import shutil
import json
import os
from pathlib import Path
from logger import logger

with open("config.json", "r") as f:
    config = json.load(f)
os.chdir(config["workdir"])

# Configuration
DATA_DIR = Path("datasets/wow-fishing-classic")
IMAGE_DIR = DATA_DIR / "images"
LABEL_DIR = DATA_DIR / "labels"
SPLITS = {"train": 0.9, "val": 0.05, "test": 0.05}
IMG_EXTENSIONS = [".jpg", ".jpeg", ".png"]

# Create destination folders
for split in SPLITS:
    (IMAGE_DIR / split).mkdir(parents=True, exist_ok=True)
    (LABEL_DIR / split).mkdir(parents=True, exist_ok=True)

# Gather all images
image_files = [f for f in IMAGE_DIR.glob("*") if f.suffix.lower() in IMG_EXTENSIONS]
random.shuffle(image_files)

# Calculate split sizes
total = len(image_files)
n_train = int(SPLITS["train"] * total)
n_val = int(SPLITS["val"] * total)
n_test = total - n_train - n_val

splits = {
    "train": image_files[:n_train],
    "val": image_files[n_train:n_train + n_val],
    "test": image_files[n_train + n_val:]
}

# Move files
for split_name, files in splits.items():
    for img_path in files:
        label_path = LABEL_DIR / img_path.with_suffix(".txt").name
        shutil.move(str(img_path), str(IMAGE_DIR / split_name / img_path.name))
        if label_path.exists():
            shutil.move(str(label_path), str(LABEL_DIR / split_name / label_path.name))

logger.info("✅ Dataset successfully split into train/val/test")


[15:17:35] [INFO ] [3433982332.py:48]  ✅ Dataset successfully split into train/val/test
